### Объявляем библиотеки

In [8]:
import psycopg2 as ps
from psycopg2.extras import DictCursor
"""from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt"""
#import codecs

'from sklearn.cluster import KMeans\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt'

### Объявляем переменные

In [3]:
PASSWORD = 'u2i6Vt6RtmQnY'
USER = 'sas'
DB = 'sas_db'
HOST = '192.168.0.71'
PORT = '5432'

dbname = 'sas_test_db'
user = 'sas'
password = 'sasTESTpassword'
host = '185.98.83.27'
port = '29001'

table = "offices_lease"

### Устанавливаем соединение

In [9]:
connection = ps.connect(dbname = dbname, user = user, password = password, host = host, port = port)
cursor = connection.cursor(cursor_factory = DictCursor)
print("Connection established with", cursor)

Connection established with <cursor object at 0x7f039498d160; closed: 0>


In [3]:
# Выгрузка базы данных частями
sql1 = """
        SELECT *
        FROM %s 
        WHERE id >= %s and id < %s
       """

# Обновить поле buildingclass
sql2 = """
        UPDATE %s 
        SET buildingclass = '%s' 
        WHERE id = %d
        """

# Получить список таблиц в базе
sql3 = """
        SELECT table_name FROM information_schema.tables WHERE table_schema NOT IN 
        ('information_schema','pg_catalog')
        """


sql4 = """
        SELECT id, fulladdress
        FROM %s
        WHERE fulladdress LIKE '%s' and isbuildingliving is Null
      """

# Получить список индексов таблицы
sql5 = """ 
        SELECT schemaname, tablename, indexname, indexdef
        FROM pg_indexes
        WHERE tablename = 'garages' AND schemaname = 'myschema'
        """

# добавить первичный ключ в таблице
sql6 = """
        ALTER TABLE %s 
        ADD CONSTRAINT id 
        PRIMARY KEY (id)
        """

# Узнать первичный ключ таблицы
# Primary key always is clastering index
sql7 = """ 
        SELECT c.column_name, c.ordinal_position
        FROM information_schema.key_column_usage AS c
        LEFT JOIN information_schema.table_constraints AS t
        ON t.constraint_name = c.constraint_name
        WHERE t.table_name = '%s' AND t.constraint_type = 'PRIMARY KEY'
        """

# Выгрузить данные из таблицы
sql8 = """
        SELECT id, price, floorscount, buildingclass, totalarea, isbuildingliving, 
                district, metrotransport, metroname, metrodistance, buildyear
        FROM %s 
        WHERE price is not Null and totalarea is not Null and fulladdress LIKE '%s' and price > 50000
        """

In [4]:
def save_db(sql, cursor, file):
    n = 50000
    with codecs.open(file, mode = 'w', encoding = 'utf-8') as f_w:
        for i in range(3):
            cursor.execute(SQL % (table, str(i*n), str((i+1)*n)))
            if i == 0: 
                col_names = [desc[0] for desc in cursor.description]
                f_w.write("\t".join(col_names))
                f_w.write("\n")
            batch = cursor.fetchall()
            for line in batch:
                f_w.write("\t".join(str(j).replace('\n', '') for j in line))
                f_w.write("\n")
                
# Получить id и classes из Result1.tsv
def get_id_classes_from_Result1():
    id_s = []
    classes = []
    with codecs.open("Result1.tsv", mode = "r", encoding = "utf-8") as f_r: 
        line = f_r.readline()
        line = f_r.readline()
        while line:
            line = line.strip('\n').split('\t')
            id_s.append(line[0].strip())
            classes.append(line[1].strip())
            line = f_r.readline()
    return id_s, classes

# Выгрузить id and classes в бд
def update_db_for_classes(id_s, classes, cursor, connection, sql):
    for i in range(len(id_s)):
        if i % 100 == 0:
            print(i, int(id_s[i]))
        cursor.execute(sql % (table, classes[i], int(id_s[i])))
        connection.commit()
        
# Получить список таблиц
def get_tables(cursor):
    cursor.execute(sql3)
    return cursor.fetchall()

# Выгрузка офисов из таблицы
def load_offices():
    SQL = """SELECT id, price, fulladdress, buildingclass, buildyear, floorscount
             FROM %s 
             WHERE fulladdress Like '%s'
          """
    cursor.execute(SQL % (table, '%Москва%'))
    with open("offices.tsv", "w") as f_w:
        f_w.write('id\tprice\taddress\tclass\tyear\tfloors\n')
        for line in cursor.fetchall(): 
            line_w = ''
            for word in line:
                line_w += str(word) + '\t'
            line_w = line_w.strip() + '\n'
            f_w.write(line_w)

# Выгрузить данные для Макса
def data_for_Maks(cursor, sql):
    cursor.execute(SQL % ("offices_lease", '%Москва%'))
    batch = cursor.fetchall()
    with open("for_Maks.tsv", "w") as f_w:
        f_w.write("id\taddress\n")
        for line in batch:
            for_write = str(line[0]) + "\t" + line[1] + "\n"
            f_w.write(for_write)

# Добавить первичный ключ к таблице
def add_primary_key_to_table(cursor, table, sql):
    cursor.execute(sql7 % table_name)
    a = cursor.fetchall()
    if not a:
        cursor.execute("SELECT * FROM %s LIMIT {size}".format(size=0) % table_name)
        offices_columns = [desc[0] for desc in cursor.description]
        if "externalid" in offices_columns:
            SQL = """ALTER TABLE %s ADD CONSTRAINT externalid PRIMARY KEY (externalid)"""
            cursor.execute(SQL % table_name)
        else:
            SQL = """ALTER TABLE %s ADD CONSTRAINT id PRIMARY KEY (id)"""
            cursor.execute(SQL % table_name)
        
# Выгрузить данные из таблицы
def load_from_db(cursor, sql):
    cursor.execute(SQL % (table, '%Москва%'))
    batch = cursor.fetchall()
    with codecs.open("50.tsv", mode = "w", encoding = "utf-8") as f_w:
        # заголовки 
        array = ['id', 'price', 'floorscount', 'buildingclass', 'totalarea', 'isbuildingliving', 
                 'district', 'metrotransport', 'metroname', 'metrodistance', 'buildyear']
        write_line = ""
        for word in array:
            write_line += word + '\t'
        write_line = write_line.strip() + "\n"
        f_w.write(write_line)

        # значения
        for line in batch:
            write_line = ""
            for word in line:
                write_line += str(word) + "\t" 
            write_line = write_line.strip() + "\n"
            f_w.write(write_line)

##### Save data base to Exel

In [5]:
cursor.execute("SELECT pg_database_size('sas_test_db')")
print (cursor.fetchone()[0])
save_db(sql1, cursor, "back_up_23-03-2020.tsv")

18145737240


###### Update data_base

In [12]:
id_s, classes = get_id_classes_from_Result1()
print (len(id_s))
update_db_for_classes(id_s, classes, cursor, connection, sql2)

###### Получить список таблиц

In [39]:
table_names = get_tables(cursor)

###### Получить список столбцов таблицы

In [67]:
name_table = "garages"
cursor.execute("SELECT * FROM %s LIMIT {size}".format(size=0) % name_table)
cols = [desc[0] for desc in cursor.description]

###### Получить список индексов таблицы

In [70]:
cursor.execute(sql5)
print(cursor.fetchall())

[]


### Делим данные 

#### Выделение улицы и дома

In [2]:
#### тренировочный ноутбук 
file_read = open("bulvar.tsv", 'r')
line = file_read.readline()
line = line.split('\t')
print (line)
file_read.close()

['1449139', 'Москва, ЦАО, район Тверской, Петровский бульвар, 15С1', 'None', 'None', 'None', '\n']


In [15]:
files_write = ["street.tsv", "prospect.tsv", "bulvar.tsv", "pereulok.tsv", "proezd.tsv", "shosse.tsv", 
               "naberejnaya.tsv", "ploschad.tsv", "typik.tsv", 'ostalnoe.tsv']
words = ["улица", "проспект", "бульвар", "переулок", "проезд", "шоссе", "набережная",  "площадь", "тупик"]
f = []
for file in files_write:
    f.append(open(file, 'w'))
with open("offices.tsv", "r") as f_r:
    line = f_r.readline()
    line = f_r.readline()
    while(line):
        for i in range(len(words)):
            if words[i] in line:
                f[i].write(line)
                break
        line = f_r.readline()
for file in f:
    file.close()

In [11]:
### Название улицы, бульвара,.., номер дома

def street(file_read, file_write, word):
    with codecs.open(file_read, mode = 'r', encoding = 'utf-8') as f_r:
        with codecs.open(file_write, mode = "w", encoding = "utf-8") as f_w:
            line = f_r.readline()
            while(line):
                line = line.strip("\n")
                line = line.split('\t')
                line_1 = line[2].split(', ')
                new_line = line[0] + '\t' + line[1] + '\t'
                for k in range (len(line_1)): 
                    if word in line_1[k]:
                        street = line_1[k]
                        break
                if k < len(line_1)-1:
                    
                    # обработка дома
                    house = line_1[k+1].lower().strip()
                    k = house.find("к")
                    c = house.find("с")
                    if k < 0 and c < 0:
                        house = house + "\t\t" 
                    elif k<0 and c>0:
                        house = house[:c] + "\t\t" + house[c+1:]
                    elif k>0 and c<0:    
                        house = house[:k] + "\t" + house[k+1:] + "\t"
                    else:
                        if k<c:
                            house = house[:k] + "\t" + house[k+1:c] + "\t" + house[c+1:]
                        else:
                            
                            house = house[:k] + "\t" + house[c+1:k] + "\t" + house[k+1:]
                            print(line)
                            print(new_line + street.replace(word, "").strip() + "\t" + house)
                    # конец обработки дома
                    
                    new_line = new_line + street.replace(word, "").strip() + "\t" + house  
                else:
                    new_line = new_line + street.replace(word, "").strip()
                for i in range(3, 6):
                    new_line = new_line + '\t' + line[i]
                f_w.write(new_line + "\n")
                line = f_r.readline()

In [12]:
# Запуск street по всем файлам

files_read = ["street.tsv", "prospect.tsv", "bulvar.tsv", "pereulok.tsv", "proezd.tsv", "shosse.tsv", 
               "naberejnaya.tsv", "ploschad.tsv", "typik.tsv"]

files_write = ["street_out.tsv", "prospect_out.tsv", "bulvar_out.tsv", "pereulok_out.tsv", "proezd_out.tsv",
               "shosse_out.tsv", "naberejnaya_out.tsv", "ploschad_out.tsv", "typik_out.tsv"]

words = ["улица", "проспект", "бульвар", "переулок", "проезд", "шоссе", "набережная",  "площадь", "тупик"]

for i in range(len(files_read)):
    street(files_read[i], files_write[i], words[i])

['1383436', '12335400.0', 'улица Вертолетчиков, Москва', 'None', 'None', '17']
1383436	12335400.0	Вертолетчиков	мо		ва
['1600357', '15457000.0', 'улица Вертолетчиков, Москва', 'None', 'None', '17']
1600357	15457000.0	Вертолетчиков	мо		ва
['1600360', '17576000.0', 'улица Вертолетчиков, Москва', 'None', 'None', '17']
1600360	17576000.0	Вертолетчиков	мо		ва
['1571366', '9577440.0', 'улица Вертолетчиков, Москва', 'None', 'None', '17']
1571366	9577440.0	Вертолетчиков	мо		ва
['146459', '264320000.00', 'Москва, ЦАО, район Хамовники, улица Ефремова, 10с1к4/1', 'b', 'None', '14']
146459	264320000.00	Ефремова	10	1	4/1
['1571361', '9240480.0', 'улица Вертолетчиков, Москва', 'None', 'None', '17']
1571361	9240480.0	Вертолетчиков	мо		ва
['1583175', '10408500.0', 'улица Вертолетчиков, Москва', 'None', 'None', '17']
1583175	10408500.0	Вертолетчиков	мо		ва
['146460', '153400000.00', 'Москва, ЦАО, район Хамовники, улица Ефремова, 10с1к4/1', 'b', 'None', '14']
146460	153400000.00	Ефремова	10	1	4/1
['1340

#### Сливаем все в один общий файл 

In [21]:
files_read = ["street_out.tsv", "prospect_out.tsv", "bulvar_out.tsv", "pereulok_out.tsv", "proezd_out.tsv",
               "shosse_out.tsv", "naberejnaya_out.tsv", "ploschad_out.tsv", "typik_out.tsv"]
words = ["улица", "проспект", "бульвар", "переулок", "проезд", "шоссе", "набережная",  "площадь", "тупик"]
file_write = "total.tsv"

In [31]:
with codecs.open(file_write, mode = 'w', encoding = 'utf-8') as f_w:
    f_w.write('id\tprice\tstreet\thouse\tstroenie\tclass\tyear\tfloors\n')
    for i  in range(len(files_read)):
        file_read = files_read[i]
        word = words[i] 
        with codecs.open(file_read, mode = "r", encoding = "utf-8") as f_r:
            line = f_r.readline()
            while line: 
                line = line.strip("\n")
                line = line.split('\t')
                line[2] = word + ' ' + line[2]
                new_line = ''
                for j in range(len(line)):
                    new_line += line[j] + '\t'
                new_line = new_line.strip() + '\n'
                f_w.write(new_line)
                line = f_r.readline()

#### Создаем массив уникальных адресов

In [28]:
array_addresses = []
array_classes = []
with open('dictionary of addresses.tsv', 'r') as f_r:
    for line in f_r:
        line = line.strip('\n').split('\t')
        array_addresses.append(line[0:4])
        array_classes.append(line[4])

In [35]:
total_addresses = []
total_id = []
with open('total.tsv', 'r') as f_r:
    line = f_r.readline()
    line = f_r.readline()
    while line:
        line = line.strip('\n').split('\t')
        total_addresses.append(line[2:6])
        total_id.append(line[0])
        line = f_r.readline()

In [33]:
print (len(array_addresses), len(total_addresses))

11700 17665


In [34]:
i = 0
for line in total_addresses:
    if line in array_addresses:
        i += 1
print (i)        

13817


In [36]:
with open("offices_Result.tsv", "w") as f_w:
    for i in range(len(total_addresses)):
        for j in range(len(array_addresses)):
            if total_addresses[i] == array_addresses[j]:
                line = total_id[i] + '\t'
                for k in total_addresses[i]:
                    line += k + '\t'
                line += array_classes[j] + '\n'
                f_w.write(line)
                break

In [15]:
result_id = []
with open("offices_Result.tsv", "r") as f_r:
    for line in f_r:
        line = line.strip('\n').split('\t')
        result_id.append(line[0])

In [16]:
with open('ostatok.tsv', 'w') as f_w:
    with open('total.tsv', 'r') as f_r:
        line = f_r.readline()
        line = f_r.readline()
        while line:
            line1 = line.strip('\n').split('\t')
            if line1[0] not in result_id:
                f_w.write(line)
            line = f_r.readline()

### Бинарный поиск

In [16]:
# бинарный поиск
def bin_search(a, value):
    mid = len(a) // 2
    low = 0
    high = len(a) - 1

    while a[mid] != value and low <= high:
        if value > a[mid]:
            low = mid + 1
        else:
            high = mid - 1
        mid = (low + high) // 2
    if low > high:
        return -1
    else:
        return mid